In [1]:
!pip install -q --upgrade rouge-score
!pip install -q --upgrade keras-hub
!pip install -q --upgrade keras # Upgrade to Keras 3.

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.1/792.1 kB 12.4 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.18.1 requires keras-hub==0.18.1, but you have keras-hub 0.20.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.18.1 requires keras-hub==0.18.1, but you have keras-hub 0.20.0 which is incompatible.


In [2]:
import keras_hub
import pathlib
import random
import keras
from keras import ops
import tensorflow as tf
import tensorflow.data as tf_data
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset as bert_vocab,
)

2025-05-10 21:06:19.364151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746911179.663794      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746911179.741120      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
BATCH_SIZE = 128
EPOCHS = 25
MAX_SEQUENCE_LENGTH = 25
ENG_VOCAB_SIZE = 20000
TR_VOCAB_SIZE = 20000
EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

In [ ]:
text_pairs = []
file_path = "/kaggle/input/veriseti/Sentence pairs in English-Turkish - 2025-05-06.tsv"
try:
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) < 4:
                continue
            _, eng, _, tr = parts[:4]
            text_pairs.append((eng.lower(), tr.lower()))
except FileNotFoundError:
    print(f"HATA: Veri dosyası bulunamadı: {file_path}")
    exit()

In [6]:
print(f"Loaded {len(text_pairs)} sentence pairs")
if not text_pairs:
    print("Hiç cümle çifti yüklenemedi.")
    exit()
print("Example:", text_pairs[0])
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = int(0.70 * len(text_pairs))
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]
print(f"{len(text_pairs)} total pairs, {len(train_pairs)} training, {len(val_pairs)} validation, {len(test_pairs)} test pairs")

Loaded 712266 sentence pairs
Example: ('those people contributed greatly to world peace.', 'bu insanlar dünya barışına çok büyük katkı sağladı.')
712266 total pairs, 498586 training, 106839 validation, 106841 test pairs


In [ ]:
def train_word_piece(text_samples, vocab_size, reserved_tokens, max_samples=200_000, sample_frac=0.5):
    n = len(text_samples)
    target = min(max_samples, int(sample_frac * n))
    text_samples_subset = random.sample(text_samples, target) if target < n else text_samples
    ds = tf_data.Dataset.from_tensor_slices(text_samples_subset)
    ds = ds.shuffle(buffer_size=len(text_samples_subset)).batch(2048).cache().prefetch(tf_data.AUTOTUNE)
    return keras_hub.tokenizers.compute_word_piece_vocabulary(ds, vocabulary_size=vocab_size, reserved_tokens=reserved_tokens)

reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]
print("Training English tokenizer...")
eng_vocab = train_word_piece([p[0] for p in train_pairs], ENG_VOCAB_SIZE, reserved_tokens)
print("Training Turkish tokenizer...")
tr_vocab = train_word_piece([p[1] for p in train_pairs], TR_VOCAB_SIZE, reserved_tokens)
print(f"English vocabulary size: {len(eng_vocab)}, Turkish vocabulary size: {len(tr_vocab)}")
eng_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(vocabulary=eng_vocab, lowercase=False)
tr_tokenizer = keras_hub.tokenizers.WordPieceTokenizer(vocabulary=tr_vocab, lowercase=False)

Training English tokenizer...


I0000 00:00:1746911227.096538      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746911227.097318      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Training Turkish tokenizer...
English vocabulary size: 6299, Turkish vocabulary size: 11091


In [ ]:
def preprocess_batch(eng, tr):
    eng = eng_tokenizer(eng)
    tr = tr_tokenizer(tr)
    eng_packer = keras_hub.layers.StartEndPacker(sequence_length=MAX_SEQUENCE_LENGTH, pad_value=eng_tokenizer.token_to_id("[PAD]"))
    tr_packer = keras_hub.layers.StartEndPacker(sequence_length=MAX_SEQUENCE_LENGTH + 1, start_value=tr_tokenizer.token_to_id("[START]"), end_value=tr_tokenizer.token_to_id("[END]"), pad_value=tr_tokenizer.token_to_id("[PAD]"))
    return {"encoder_inputs": eng_packer(eng), "decoder_inputs": tr_packer(tr)[:, :-1]}, tr_packer(tr)[:, 1:]

def make_dataset(pairs):
    eng_texts, tr_texts = zip(*pairs)
    dataset = tf_data.Dataset.from_tensor_slices((list(eng_texts), list(tr_texts)))
    return dataset.batch(BATCH_SIZE).map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE).shuffle(2048).prefetch(tf_data.AUTOTUNE).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
print("\nDataset check:")
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}, inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}, targets.shape: {targets.shape}')


Dataset check:
inputs["encoder_inputs"].shape: (128, 25), inputs["decoder_inputs"].shape: (128, 25), targets.shape: (128, 25)


In [ ]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x_enc = keras_hub.layers.TokenAndPositionEmbedding(ENG_VOCAB_SIZE, MAX_SEQUENCE_LENGTH, EMBED_DIM)(encoder_inputs)
encoder_outputs = keras_hub.layers.TransformerEncoder(INTERMEDIATE_DIM, NUM_HEADS)(inputs=x_enc)
encoder = keras.Model(encoder_inputs, encoder_outputs, name="encoder")

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")
x_dec = keras_hub.layers.TokenAndPositionEmbedding(TR_VOCAB_SIZE, MAX_SEQUENCE_LENGTH, EMBED_DIM)(decoder_inputs)
decoder_mid = keras_hub.layers.TransformerDecoder(INTERMEDIATE_DIM, NUM_HEADS)(x_dec, encoded_seq_inputs)
decoder_mid_dropout = keras.layers.Dropout(0.5)(decoder_mid)
decoder_outputs_final = keras.layers.Dense(TR_VOCAB_SIZE, activation="softmax")(decoder_mid_dropout)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs_final, name="decoder")

encoder_output_tensor = encoder(encoder_inputs)
decoder_output_tensor = decoder([decoder_inputs, encoder_output_tensor])
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_output_tensor, name="transformer")
transformer.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder (Functional)      │ (None, None, 256)      │      6,441,472 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder (Functional)      │ (None, None, 20000)    │     11,845,152 │ decoder_inputs[0][0],  │
│                           │                        │                │ encoder[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 18,286,624 (69.76 MB)

 Trainable params: 18,286,624 (69.76 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
transformer.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
print("\nStarting training...")
history = transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)
print("Training finished.")


Starting training...
Epoch 1/25


I0000 00:00:1746911488.968717     118 service.cc:148] XLA service 0x7d31041b5770 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746911488.969891     118 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1746911488.969914     118 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
W0000 00:00:1746911489.362329     118 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1746911490.226999     118 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/3896 ━━━━━━━━━━━━━━━━━━━━ 23:04:17 21s/step - accuracy: 0.0000e+00 - loss: 9.8446

I0000 00:00:1746911500.640588     118 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3192/3896 ━━━━━━━━━━━━━━━━━━━━ 1:16 108ms/step - accuracy: 0.7821 - loss: 1.7830

W0000 00:00:1746911845.865232     118 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


3896/3896 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.7877 - loss: 1.7031

W0000 00:00:1746911932.923616     119 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1746911940.468368     118 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


3896/3896 ━━━━━━━━━━━━━━━━━━━━ 475s 117ms/step - accuracy: 0.7878 - loss: 1.7030 - val_accuracy: 0.8560 - val_loss: 0.8604
Epoch 2/25
3896/3896 ━━━━━━━━━━━━━━━━━━━━ 441s 113ms/step - accuracy: 0.8555 - loss: 0.8798 - val_accuracy: 0.8750 - val_loss: 0.7188
Epoch 3/25
3896/3896 ━━━━━━━━━━━━━━━━━━━━ 441s 113ms/step - accuracy: 0.8704 - loss: 0.7723 - val_accuracy: 0.8833 - val_loss: 0.6724
Epoch 4/25
3896/3896 ━━━━━━━━━━━━━━━━━━━━ 441s 113ms/step - accuracy: 0.8779 - loss: 0.7262 - val_accuracy: 0.8870 - val_loss: 0.6525
Epoch 5/25
3896/3896 ━━━━━━━━━━━━━━━━━━━━ 438s 112ms/step - accuracy: 0.8832 - loss: 0.6922 - val_accuracy: 0.8911 - val_loss: 0.6307
Epoch 6/25
3896/3896 ━━━━━━━━━━━━━━━━━━━━ 436s 112ms/step - accuracy: 0.8876 - loss: 0.6644 - val_accuracy: 0.8932 - val_loss: 0.6185
Epoch 7/25
3896/3896 ━━━━━━━━━━━━━━━━━━━━ 435s 112ms/step - accuracy: 0.8914 - loss: 0.6407 - val_accuracy: 0.8950 - val_loss: 0.6096
Epoch 8/25
3896/3896 ━━━━━━━━━━━━━━━━━━━━ 435s 112ms/step - accuracy: 0.8

In [12]:
def decode_sequences(input_sentences_list):
    batch_size = len(input_sentences_list)
    encoder_tokens_ragged = eng_tokenizer(input_sentences_list)
    eng_packer = keras_hub.layers.StartEndPacker(sequence_length=MAX_SEQUENCE_LENGTH, pad_value=eng_tokenizer.token_to_id("[PAD]"))
    encoder_input_data = eng_packer(encoder_tokens_ragged)
    start_id = tr_tokenizer.token_to_id("[START]")
    pad_id_tr = tr_tokenizer.token_to_id("[PAD]")
    prompt = tf.concat([tf.fill([batch_size, 1], start_id), tf.fill([batch_size, MAX_SEQUENCE_LENGTH - 1], pad_id_tr)], axis=1)
    prompt = ops.cast(prompt, dtype="int64")

    def next_fn(current_decoder_input, cache, index):
        logits = transformer([encoder_input_data, current_decoder_input])[:, index - 1, :]
        return logits, None, cache

    greedy_sampler = keras_hub.samplers.GreedySampler()
    generated_tokens = greedy_sampler(
        next_fn,
        prompt,
        stop_token_ids=[tr_tokenizer.token_to_id("[END]")],
        index=1,
    )
    generated_sentences = tr_tokenizer.detokenize(generated_tokens)
    return [s.numpy().decode("utf-8") if isinstance(s, tf.Tensor) else s for s in generated_sentences]

In [13]:
print("\n--- Translation Examples ---")
if test_pairs:
    test_eng_texts = [pair[0] for pair in test_pairs]
    if test_eng_texts:
        for i in range(min(10, len(test_eng_texts))):
            input_sentence = random.choice(test_eng_texts)
            translations = decode_sequences([input_sentence])
            translated = translations[0].replace("[PAD]", "").replace("[START]", "").replace("[END]", "").strip()
            print(f"** Example {i+1} **\nSRC: {input_sentence}\nTRG: {translated}\n")
    else:
        print("Test için İngilizce cümle bulunamadı.")
else:
    print("`test_pairs` listesi bulunamadı veya boş. Çeviri örnekleri atlanıyor.")


--- Translation Examples ---
** Example 1 **
SRC: latin word order is free, but not arbitrary.
TRG: latincederek ücretsizdir ama aksine , ama aksine .

** Example 2 **
SRC: i might try to do that.
TRG: bunu yapmaya çalışabilirim .

** Example 3 **
SRC: i know tom is very amusing.
TRG: tom ' un çok komik olduğunu biliyorum .

** Example 4 **
SRC: our music teacher advised me to visit vienna.
TRG: müzik öğretmenimiz beni ziyaret etmemi tavsiye etti .

** Example 5 **
SRC: none of my friends speak french.
TRG: arkadaşlarımdan hiçbiri fransızca konuşmaz .

** Example 6 **
SRC: how many teeth does a horse have?
TRG: kaç tane diş fer var ?

** Example 7 **
SRC: thank you again for your kind assistance.
TRG: senin nazik yardımını tekrar teşekkürler .

** Example 8 **
SRC: he's on his last legs.
TRG: o , son ayağında , .

** Example 9 **
SRC: i'm anxious to know the results of the blood test.
TRG: kan testin sonuçları bilmekten endişeliyim .

** Example 10 **
SRC: tom was angry, but mary wasn

## Conclusion of Qualitative Assessment (Task d):

- The model performs quite well on simple and direct sentences (Examples 2, 3, 5, 10).
- There are minor structural or word-choice errors in some sentences, but the main idea is generally understandable (Examples 4, 7, 9).
- The model struggles with idioms (Example 8) or more complex structures/wordplay (Examples 1, 6), tending towards word-for-word translations. The mistranslation of "free" as "ücretsizdir" (free of charge) instead of "serbest" (free/unconstrained) (Example 1), or the generation of a nonsensical word like "fer" for "horse" (Example 6), indicates situations where the model may not fully grasp the context or has encountered insufficient data for those specific cases.

In [14]:
print("\n--- Evaluation on Test Set ---")
rouge_1_eval = keras_hub.metrics.RougeN(order=1, name='rouge_1_eval')
rouge_2_eval = keras_hub.metrics.RougeN(order=2, name='rouge_2_eval')
rouge_l_eval = keras_hub.metrics.RougeL(name='rouge_l_eval')

if test_pairs:
    num_eval_samples = min(len(test_pairs), 300)
    print(f"Evaluating on {num_eval_samples} test samples...")
    for i, (input_sentence, reference_sentence) in enumerate(test_pairs[:num_eval_samples]):
        if i % 50 == 0 and i > 0: print(f"  Evaluated {i}/{num_eval_samples} samples...")
        translations = decode_sequences([input_sentence])
        translated = translations[0].replace("[PAD]", "").replace("[START]", "").replace("[END]", "").strip()
        rouge_1_eval.update_state(reference_sentence, translated)
        rouge_2_eval.update_state(reference_sentence, translated)
        rouge_l_eval.update_state(reference_sentence, translated)
    print("Evaluation finished.")

    r1_result = rouge_1_eval.result()
    r2_result = rouge_2_eval.result()
    rl_result = rouge_l_eval.result()


--- Evaluation on Test Set ---
Evaluating on 300 test samples...
  Evaluated 50/300 samples...
  Evaluated 100/300 samples...
  Evaluated 150/300 samples...
  Evaluated 200/300 samples...
  Evaluated 250/300 samples...
Evaluation finished.


In [15]:
print("\n--- ROUGE Scores ---")
print(f"ROUGE-1 Scores:\n  Precision: {r1_result['precision'].numpy():.4f}\n  Recall:    {r1_result['recall'].numpy():.4f}\n  F1 Score:  {r1_result['f1_score'].numpy():.4f}")
print(f"ROUGE-2 Scores:\n  Precision: {r2_result['precision'].numpy():.4f}\n  Recall:    {r2_result['recall'].numpy():.4f}\n  F1 Score:  {r2_result['f1_score'].numpy():.4f}")
print(f"ROUGE-L Scores:\n  Precision: {rl_result['precision'].numpy():.4f}\n  Recall:    {rl_result['recall'].numpy():.4f}\n  F1 Score:  {rl_result['f1_score'].numpy():.4f}")


--- ROUGE Scores ---
ROUGE-1 Scores:
  Precision: 0.6922
  Recall:    0.6935
  F1 Score:  0.6764
ROUGE-2 Scores:
  Precision: 0.5176
  Recall:    0.5178
  F1 Score:  0.5128
ROUGE-L Scores:
  Precision: 0.6803
  Recall:    0.6815
  F1 Score:  0.6746
